### Just change the code to predict oof feature individually for each type 

In [1]:
import numpy as np
import pandas as pd
import feather
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import gc

import sys
sys.path.append("./../")
from utils import reduce_mem_usage

In [2]:
#実際予測時にはその化合物のデータは使えないので、厳密にはGroupKfoldで化合物ごとに分けるべき

In [3]:
features = [
        "Atom",
        "AtomPosition",
        "AtomDistance",
        "CouplingType",
        "AtomEnvironment",
        "AtomNeighbors",
        "BruteForce",
        "DistanceFromClosest",
        "ElectroNegFromClosest",
        "ACSF",
        "MullikenChargePred",
        "OpenBabelBasic",
        "CosineDistance",
        "PotentialPred"]

In [4]:
X_train = pd.concat([feather.read_dataframe("../features/" + feature + "_train.feather") for feature in features], axis=1)
X_test = pd.concat([feather.read_dataframe("../features/" + feature + "_test.feather") for feature in features], axis=1)

In [5]:
X_train.shape, X_test.shape

((4658147, 486), (2505542, 486))

In [6]:
X_train.head()

,atom_0,atom_1,x_0,x_1,y_0,y_1,z_0,z_1,atom_distance,dist_inv_0_sum,...,cos_c0_c1,cos_f0_f1,cos_center0_center1,cos_c0,cos_c1,cos_f0,cos_f1,cos_center0,cos_center1,potential_oof
0,H,C,0.002150,-0.012698,-0.006031,1.085804,0.001976,0.008001,1.091953,1.297179,...,0.333335,-0.816502,-0.649313,-1.000000,-0.333335,-0.816502,1.000000,-1.000000,0.649319,-69.063046
1,H,H,0.002150,1.011731,-0.006031,1.463751,0.001976,0.000277,1.783120,1.297179,...,-0.333287,0.000016,-0.333304,-0.816483,0.816482,-0.499994,0.499995,-0.816487,0.816488,-69.063046
2,H,H,0.002150,-0.540815,-0.006031,1.447527,0.001976,-0.876644,1.783147,1.297179,...,-0.333335,-0.000016,-0.333335,-0.816498,0.816496,-0.500002,0.500018,-0.816496,0.816499,-69.063046
3,H,H,0.002150,-0.523814,-0.006031,1.437933,0.001976,0.906397,1.783157,1.297179,...,-0.333347,-1.000000,-0.333349,-0.816502,0.816500,-1.000000,1.000000,-0.816500,0.816503,-69.063046
4,H,C,1.011731,-0.012698,1.463751,1.085804,0.000277,0.008001,1.091952,1.120798,...,0.333352,-0.000028,-0.499068,-1.000000,-0.333352,-0.816503,-0.333287,-1.000000,0.499076,-69.063046


In [7]:
contri = feather.read_dataframe("../data/input/scalar_coupling_contributions.feather")

In [8]:
contri.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [9]:
# keep nan as is
#coupling_types = X_train['type']
categorical_cols = list(X_train.columns[X_train.dtypes == object])


for col in categorical_cols:    
    print(f'Starting {col}')
    uniques = list(X_train[col].unique())
    if None in uniques:
        uniques.remove(None)
    mapping = dict(zip(uniques, range(1, len(uniques)+1)))
    X_train[col] = X_train[col].map(mapping)
    X_test[col] = X_test[col].map(mapping)

Starting atom_0
Starting atom_1
Starting type
Starting a1_is_aromatic
Starting a1_in_ring
Starting a1_in_ring3
Starting a1_in_ring4
Starting a1_in_ring5
Starting a1_in_ring6
Starting a1_in_ring7
Starting a1_in_ring8
Starting bond_atom
Starting tertiary_atom_0
Starting tertiary_atom_1
Starting tertiary_atom_2
Starting tertiary_atom_3
Starting tertiary_atom_4
Starting tertiary_atom_5
Starting tertiary_atom_6
Starting tertiary_atom_7
Starting tertiary_atom_8
Starting tertiary_atom_9


In [10]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Memory usage of dataframe is 16312.33 MB
Memory usage after optimization is: 8538.21 MB
Decreased by 47.7%
Memory usage of dataframe is 8764.58 MB
Memory usage after optimization is: 4575.84 MB
Decreased by 47.8%


In [11]:
gc.collect()

0

In [12]:
train_types = X_train['type'].values
test_types = X_test['type'].values

In [13]:
oof_train = pd.DataFrame()
oof_test = pd.DataFrame()


for col in ['fc', 'sd', 'pso', 'dso']:
    
    target = contri[col]
    
    oof_type = np.zeros(len(X_train))
    pred_type = np.zeros(len(X_test))
    
    for cur_type in np.unique(train_types):
        cur_type_idx_train = (train_types == cur_type)
        cur_type_idx_test = (test_types == cur_type)

        cur_type_train = X_train.iloc[cur_type_idx_train]
        cur_type_test = X_test.iloc[cur_type_idx_test]
        cur_type_target = target.iloc[cur_type_idx_train]
        cur_type_mols = contri['molecule_name'].iloc[cur_type_idx_train]
        
    
        kf = GroupKFold(n_splits=2)
    
        oof = np.zeros(len(cur_type_train))
        pred = np.zeros(len(cur_type_test))
        
        
        SEED = 42
        NUM_ROUNDS = 10000
        
        params = {
                    "num_leaves": 100,
                    "min_data_in_leaf": 100,
                    "objective": "regression",
                    "max_depth": 10,
                    "learning_rate": 0.2,
                    "boosting_type": "gbdt",
                    "subsample_freq": 1,
                    "subsample": 0.9,
                    "metric": "mae",
                    "reg_alpha": 0.1,
                    "reg_lambda": 0.3, 
                    "colsample_bytree": 0.9
                    }

        for train_idx, val_idx in kf.split(cur_type_train, groups=cur_type_mols):
            train_data = lgb.Dataset(cur_type_train.iloc[train_idx], label=cur_type_target.iloc[train_idx], categorical_feature=categorical_cols)
            val_data = lgb.Dataset(cur_type_train.iloc[val_idx], label=cur_type_target.iloc[val_idx], categorical_feature=categorical_cols)
            clf = lgb.train(params, train_data, NUM_ROUNDS, valid_sets=[train_data, val_data],
                            verbose_eval=1000, early_stopping_rounds=100)
            oof[val_idx] = clf.predict(cur_type_train.iloc[val_idx], num_iteration=clf.best_iteration)
            pred += clf.predict(cur_type_test, num_iteration=clf.best_iteration) / kf.n_splits

        oof_type[cur_type_idx_train] = oof
        pred_type[cur_type_idx_test] = pred
        
        print(f'{cur_type} for {col} done!')
        
    oof_train[col] = oof_type
    oof_test[col] = pred_type

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.403637	valid_1's l1: 0.763418
[2000]	training's l1: 0.233837	valid_1's l1: 0.72765
[3000]	training's l1: 0.147417	valid_1's l1: 0.716215
[4000]	training's l1: 0.0967549	valid_1's l1: 0.711107
[5000]	training's l1: 0.065312	valid_1's l1: 0.708594
[6000]	training's l1: 0.0450368	valid_1's l1: 0.707206
[7000]	training's l1: 0.0315582	valid_1's l1: 0.706468
[8000]	training's l1: 0.0224226	valid_1's l1: 0.705996
[9000]	training's l1: 0.0160858	valid_1's l1: 0.705731
[10000]	training's l1: 0.0117098	valid_1's l1: 0.705565
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.0117098	valid_1's l1: 0.705565
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.403257	valid_1's l1: 0.767631
[2000]	training's l1: 0.233933	valid_1's l1: 0.732686
[3000]	training's l1: 0.147655	valid_1's l1: 0.72111
[4000]	training's l1: 0.0973058	valid_1's l1: 0.716081
[5000]	t

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.062919	valid_1's l1: 0.184902
[2000]	training's l1: 0.0278901	valid_1's l1: 0.180833
[3000]	training's l1: 0.0138185	valid_1's l1: 0.179842
[4000]	training's l1: 0.00741878	valid_1's l1: 0.179469
[5000]	training's l1: 0.00429224	valid_1's l1: 0.179339
[6000]	training's l1: 0.00267842	valid_1's l1: 0.179278
[7000]	training's l1: 0.00179722	valid_1's l1: 0.179248
[8000]	training's l1: 0.001288	valid_1's l1: 0.179231
[9000]	training's l1: 0.000976825	valid_1's l1: 0.179223
[10000]	training's l1: 0.000780628	valid_1's l1: 0.179217
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000780628	valid_1's l1: 0.179217
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.0637084	valid_1's l1: 0.184399
[2000]	training's l1: 0.0282191	valid_1's l1: 0.180356
[3000]	training's l1: 0.0139963	valid_1's l1: 0.179307
[4000]	training's l1: 0.00751977	valid_1's l1:

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


...

Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00904947	valid_1's l1: 0.0178458
[2000]	training's l1: 0.00528707	valid_1's l1: 0.0172889
[3000]	training's l1: 0.00343748	valid_1's l1: 0.0170968
[4000]	training's l1: 0.0023961	valid_1's l1: 0.0170262
[5000]	training's l1: 0.00175512	valid_1's l1: 0.0169916
[6000]	training's l1: 0.00134219	valid_1's l1: 0.016974
[7000]	training's l1: 0.00106523	valid_1's l1: 0.0169636
[8000]	training's l1: 0.000872996	valid_1's l1: 0.0169568
[9000]	training's l1: 0.000735777	valid_1's l1: 0.0169533
[10000]	training's l1: 0.000637138	valid_1's l1: 0.0169504
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.000637138	valid_1's l1: 0.0169504
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00896539	valid_1's l1: 0.0177826
[2000]	training's l1: 0.00522798	valid_1's l1: 0.017199
[3000]	training's l1: 0.00340628	valid_1's l1: 0.0170266
[4000]	training's l1: 0.0

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00578	valid_1's l1: 0.00790021
[2000]	training's l1: 0.0041862	valid_1's l1: 0.00735918
[3000]	training's l1: 0.00327039	valid_1's l1: 0.00714012
[4000]	training's l1: 0.00264957	valid_1's l1: 0.0070235
[5000]	training's l1: 0.00220272	valid_1's l1: 0.00695905
[6000]	training's l1: 0.00186168	valid_1's l1: 0.006914
[7000]	training's l1: 0.00159501	valid_1's l1: 0.00688242
[8000]	training's l1: 0.00138429	valid_1's l1: 0.0068626
[9000]	training's l1: 0.0012133	valid_1's l1: 0.00684773
[10000]	training's l1: 0.00107418	valid_1's l1: 0.00683665
Did not meet early stopping. Best iteration is:
[10000]	training's l1: 0.00107418	valid_1's l1: 0.00683665
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00578634	valid_1's l1: 0.00786793
[2000]	training's l1: 0.00419983	valid_1's l1: 0.00733569
[3000]	training's l1: 0.00327559	valid_1's l1: 0.00711643
[4000]	training's l1:

C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\kohei\Anaconda3\lib\site-packages\lightgbm\basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.000749342	valid_1's l1: 0.00252224
[2000]	training's l1: 0.000565829	valid_1's l1: 0.00250063
[3000]	training's l1: 0.00050431	valid_1's l1: 0.00249345
Early stopping, best iteration is:
[3446]	training's l1: 0.00048888	valid_1's l1: 0.0024917
Training until validation scores don't improve for 100 rounds.
[1000]	training's l1: 0.00074247	valid_1's l1: 0.00250675
[2000]	training's l1: 0.000559189	valid_1's l1: 0.0024823
[3000]	training's l1: 0.000499185	valid_1's l1: 0.00247536
Early stopping, best iteration is:
[3444]	training's l1: 0.000483902	valid_1's l1: 0.00247356
8 for dso done!


In [14]:
oof_train.shape, oof_test.shape

((4658147, 4), (2505542, 4))

In [15]:
oof_train.isnull().sum()

fc     0
sd     0
pso    0
dso    0
dtype: int64

In [16]:
oof_train.to_feather('../features/ScalarCouplingContributionsOof4_train.feather')
oof_test.to_feather('../features/ScalarCouplingContributionsOof4_test.feather')